##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>   </td>
</table>

このガイドでは簡単な導入として、[Keras](https://www.tensorflow.org/guide/keras/overview) を使って次のことを行います。

1. 画像を分類するニューラルネットワークを構築する。
2. そのニューラルネットワークをトレーニングする。
3. 最後に、モデルの精度を評価する。
4. モデルの精度を評価します。

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

## TensorFlow をセットアップする

まず、TensorFlow をプログラムにインポートします。

In [2]:
import tensorflow as tf

2022-08-08 19:21:47.646123: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2022-08-08 19:21:48.346837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory
2022-08-08 19:21:48.347087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory
2022-08-08 19:21:48.347099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


[Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) ではなく独自の開発環境で手順を実行している場合は、開発用 TensorFlow のセットアップについて、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

注意: 独自の開発環境を使用している場合は、TensorFlow 2 パッケージをインストールする際に、最新の `pip` にアップグレード済みであることを確認してください。詳細は、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

## データセットを読み込む

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)を読み込んで準備します。サンプルデータを整数から浮動小数点数に変換します。

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 機械学習モデルを構築する

層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

各サンプルについて、モデルは "[logits](https://developers.google.com/machine-learning/glossary#logits)" または "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" スコアのベクトルをクラスごとに返します。

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.05896852,  0.9313769 , -0.73717177, -0.4086499 , -0.5739427 ,
         0.46960253, -0.09548981,  0.01933566, -0.20799409, -0.00790808]],
      dtype=float32)

`tf.nn.softmax` 関数は、クラスごとにこれらのロジットを "probabilities" に変換します。 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.09972002, 0.23859671, 0.04498037, 0.06247385, 0.05295567,
        0.15035519, 0.08544801, 0.09584512, 0.07635579, 0.09326918]],
      dtype=float32)

注意: この `tf.nn.softmax` をネットワークの最後のレイヤーのアクティベーション関数として組み込むことも可能です。こうすることでモデルの出力をより直接的に解釈可能にすることもできますが、softmax 出力を使用する場合、すべてのモデルに対して正確で数値的に安定した損失計算を提供することは不可能であるため、この方法は推奨されません。 

`losses.SparseCategoricalCrossentropy` 損失は、ロジットのベクトルと `True` インデックスを取り、各サンプルのスカラー損失を返します。

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、True のクラスの負の対数確率に等しくなります。モデルが正しいクラスであることが確実な場合はゼロです。

トレーニングされていないこのモデルでは、ランダムに近い確率（クラス当たり 1/10）が得られるため、最初の損失は `-tf.math.log(1/10) ~= 2.3` に近くなります。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

1.8947548

トレーニングを開始する前に、Keras `Model.compile` を使用してモデルの構成とコンパイルを行います。[`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) クラスを `adam` に、`loss` を前に定義した `loss_fn` 関数に設定し、`metrics` パラメータを `accuracy` に設定して評価するモデルの指標を指定します。

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## モデルをトレーニングして評価する

損失を最小限に抑えられるようにモデルのパラメータを `Model.fit` メソッドで調整します。 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


   1/1875 [..............................] - ETA: 10:25 - loss: 2.4618 - accuracy: 0.0625

  29/1875 [..............................] - ETA: 3s - loss: 1.7139 - accuracy: 0.4731   

  59/1875 [..............................] - ETA: 3s - loss: 1.3158 - accuracy: 0.6043

  89/1875 [>.............................] - ETA: 3s - loss: 1.0957 - accuracy: 0.6710

...

1786/1875 [===========================>..] - ETA: 0s - loss: 0.0729 - accuracy: 0.9769

1814/1875 [============================>.] - ETA: 0s - loss: 0.0730 - accuracy: 0.9769

1842/1875 [============================>.] - ETA: 0s - loss: 0.0728 - accuracy: 0.9769

1871/1875 [============================>.] - ETA: 0s - loss: 0.0727 - accuracy: 0.9770

1875/1875 [==============================] - 3s 2ms/step - loss: 0.0726 - accuracy: 0.9770


`Model.evaluate` メソッドは通常、[検証セット](https://developers.google.com/machine-learning/glossary#validation-set)が[テストセット](https://developers.google.com/machine-learning/glossary#test-set)でモデルのパフォーマンスをチェックします。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0727 - accuracy: 0.9777 - 568ms/epoch - 2ms/step


[0.07274919748306274, 0.9776999950408936]

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

モデルが確率を返すようにするには、トレーニング済みのモデルをラップして、それに softmax を接続することができます。

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.5685875e-07, 2.3471394e-08, 4.1818867e-06, 7.0784015e-05,
        2.5523697e-10, 5.2577459e-08, 2.5774269e-14, 9.9992406e-01,
        6.4098380e-08, 7.0080222e-07],
       [2.2749015e-07, 2.5114664e-04, 9.9971670e-01, 3.1221844e-05,
        1.1798841e-14, 7.4317711e-07, 2.4693703e-09, 1.9629888e-11,
        2.9157308e-08, 3.5419436e-12],
       [2.1835449e-07, 9.9951446e-01, 1.3223765e-05, 2.3281484e-06,
        7.8297862e-06, 6.3806629e-06, 3.7711166e-06, 4.3334873e-04,
        1.8290575e-05, 1.0623557e-07],
       [9.9998844e-01, 5.2491025e-13, 6.5724494e-06, 8.6036334e-09,
        3.2563381e-07, 7.1521504e-07, 6.6615547e-07, 8.8499803e-07,
        1.7735917e-09, 2.4320404e-06],
       [4.6543541e-06, 2.1472955e-08, 4.7988442e-06, 3.4825671e-08,
        9.9699819e-01, 2.7171569e-07, 1.1828049e-05, 8.7052074e-05,
        9.6883355e-07, 2.8922674e-03]], dtype=float32)>

## まとめ

おめでとうございます！事前構築済みのデータセットと [Keras](https://www.tensorflow.org/guide/keras/overview) API を使用して、機械学習モデルをトレーニングしました。

Keras のその他の使用例については、[チュートリアル](https://www.tensorflow.org/tutorials/keras/)をご覧ください。Keras を使ったモデルの構築に関してさらに学習するには、[ガイド](https://www.tensorflow.org/guide/keras)をお読みください。データの読み込みと準備についてさらに学習するには、[画像データの読み込み](https://www.tensorflow.org/tutorials/load_data/images)または [CSV データの読み込み](https://www.tensorflow.org/tutorials/load_data/csv)に関するチュートリアルをご覧ください。
